In [1]:
import spacy
import pdfminer 
import re 
import os 
import pandas as pd 

import pdf2txt

In [2]:
def convert_pdf(f):
    output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt"
    output_filepath = os.path.join("output/txt/", output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath])
    print(output_filepath + " saved!")
    return open(output_filepath).read()

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
result_dict = {'name' : [], 'phone' : [], 'email' : [], 'skills' : []}
names = []
phones = []
emails = []
skills = []

In [7]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ == "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Done!")

In [8]:
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading.....' + file)
        txt = convert_pdf(os.path.join('resumes/',file))
        parse_content(txt)

Reading.....Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved!
Alisson Parker-Wright
alli1414parks@mail.com
Done!
Reading.....AshleyMilesCV.pdf
output/txt/AshleyMilesCV.txt saved!
Ashley Miles
ashleymiles@memail.com
Done!
Reading.....John DominicCV.pdf
output/txt/John DominicCV.txt saved!
John Dominic
johndominic@mail.com
Done!


In [11]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills

result_df = pd.DataFrame(result_dict)
result_df

name           phone                   email  \
0  Alisson Parker-Wright  ['8569878511']  alli1414parks@mail.com   
1           Ashley Miles  ['6592251422']  ashleymiles@memail.com   
2           John Dominic  ['7877756411']    johndominic@mail.com   

                          skills  
0  {'java', 'tableau', 'python'}  
1             {'tableau', 'sql'}  
2   {'java', 'hadoop', 'python'}

In [12]:
result_df.to_csv('output/csv/parsed_resumes.csv')